In [1]:
%%capture --no-display
from dask.distributed import Client
client = Client(address = 'tcp://192.168.62.207:8786')
client

Client Scheduler: tcp://192.168.62.207:8786 Dashboard: http://192.168.62.207:8787/status,Cluster Workers: 10 Cores: 10 Memory: 673.47 GB


In [2]:
import takco
# fnames = '/export/scratch1/home/kruit/scratch/output/tabel-small-4/0-reshape/*.jsonl'
fnames = 'hdfs:///user/kruit/output/tabel-all2/0-reshape/*'
tables = takco.DaskHashBag(client=client).load(fnames).persist()
tables.bag.count().compute()

1499863

In [5]:
fnames2 = 'hdfs:///user/kruit/output/tabel-all2/0-reshape/03*'
tables2 = takco.DaskHashBag(client=client).load(fnames2).persist()
from collections import Counter
Counter(tables2.bag.groupby(lambda t: t.headerId).map(lambda x: len(x[1]))).most_common(3)

[(1, 13510), (2, 2092), (3, 771)]

In [7]:
Counter(dict(tables2.bag.groupby(lambda t: t.head).map(lambda x: (x[0], len(x[1]))))).most_common(3)

[(((LinkedString('', links=()),
    LinkedString('Preceded by', links=()),
    LinkedString('', links=()),
    LinkedString('Succeeded by', links=())),),
  429),
 (((LinkedString('', links=()), LinkedString('Source', links=())),), 167),
 (((LinkedString('Preceded by', links=()),),), 160)]

In [10]:
%%time
import takco
unions = tables.bag.groupby(lambda t: t.headerId).map(lambda x: takco.Table.concat(x[1]))
unions.count().compute()

CPU times: user 1min 35s, sys: 5.21 s, total: 1min 40s
Wall time: 5min 17s


In [21]:
%%time
from collections import Counter
import takco

# headers = tables.bag.map(lambda t:t.head).frequencies()
key = lambda t: t.head
binop = lambda total, t: total + len(t.body)
combine = lambda a,b: a+b
headers = tables.bag.foldby(key, binop=binop, initial=0, combine = combine).compute()

for head, cnt in Counter(dict(headers)).most_common(10):
    print(cnt)
    display(takco.Table(head=head))

492429


,0,1,2
,Home \ Away [1 ],_Variable,_Value


325847


,0,1,2,3,4
,Pos,Driver,Pts,_Variable,_Value


202195


...

152989


,0,1,2,3,4
,Club,Season,_Variable,Apps,Goals


146354


,0,1,2,3,4,5
,,Pos,Driver,Pts,_Variable,_Value


CPU times: user 13.5 s, sys: 1.25 s, total: 14.7 s
Wall time: 1min 23s


In [5]:
biggest = sorted(tables, key=lambda t: -len(t.body))
takco.preview(biggest)

?,0,1,2,3
,Season,European Competitions,_Variable,_Value
,Total,,Greek Super League,Total
,Total,,Greek Super League,Total
,Total,,Greek Super League,Total
,Total,,Greek Super League,Total
,Total,,Greek Super League,Total
?,0,1,2,3
,Rank,Country,_Variable,
,1.,Qatar,1990,25.2
,1.,Qatar,1991,36.7
